# Описание проекта

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Задачи 

1. Загрузить данные
2. Ответить на вопросы задания:
- Сколько книг вышло после 1 января 2000 года?
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок;
3. Вывод по заданию

## Шаг 1. Загрузка данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [4]:
# первые пять строк таблицы books
query_books = ''' SELECT *
        FROM books
        LIMIT 5
    '''
books = pd.io.sql.read_sql(query_books, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
# первые пять строк таблицы authors
query_authors = ''' SELECT *
        FROM authors
        LIMIT 5
    '''
authors = pd.io.sql.read_sql(query_authors, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
# первые пять строк таблицы publishers
query_publishers = ''' SELECT *
        FROM publishers
        LIMIT 5
    '''
publishers = pd.io.sql.read_sql(query_publishers, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
# первые пять строк таблицы ratings
query_ratings = ''' SELECT *
        FROM ratings
        LIMIT 5
    '''
ratings = pd.io.sql.read_sql(query_ratings, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
# первые пять строк таблицы reviews
query_reviews = ''' SELECT *
        FROM reviews
        LIMIT 5
    '''
reviews = pd.io.sql.read_sql(query_reviews, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Данные загружены, приступаем к следующему шагу

## Шаг 2. Ответы на вопросы задания

In [9]:
# посчитаем сколько книг было выпущено после 1 января 2000 года
books_count = '''SELECT 
COUNT (book_id) AS cnt
FROM books
WHERE publication_date > '2000-01-01'
'''
books_count = pd.io.sql.read_sql(books_count, con = engine)
books_count

,cnt
0,819


После 1 января 2000 года было выпущено 819 книг

In [10]:
# Для каждой книги посчитаем количество обзоров и среднюю оценку;
review_rating = '''SELECT
        ratings.book_id AS book_id,
        COUNT(reviews.text) AS total_reviews,
        AVG(ratings.rating) AS mean_rating
        FROM
            ratings
        LEFT JOIN reviews ON 
            (reviews.book_id = ratings.book_id
            AND reviews.username = ratings.username)
        GROUP BY 
            ratings.book_id
        ORDER BY 
            ratings.book_id
        LIMIT 10
'''
review_rating = pd.io.sql.read_sql(review_rating, con = engine)
review_rating

,book_id,total_reviews,mean_rating
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
5,6,4,3.500000
6,7,3,3.000000
7,8,3,4.000000
8,9,2,4.250000
9,10,3,3.200000


Для каждой книги посчитаны количество отзывов и средняя оценка

In [14]:
#Определим издательство, которое выпустило наибольшее число книг толще 50 страниц
publishers_books = '''SELECT
        publishers.publisher AS publisher,
        COUNT(books.book_id) AS total_books
        FROM
            books
        LEFT JOIN publishers ON 
            publishers.publisher_id = books.publisher_id
        GROUP BY 
            publishers.publisher
        HAVING
            SUM(books.num_pages) > 50
        ORDER BY 
            total_books DESC
        LIMIT 10
'''
publishers_books = pd.io.sql.read_sql(publishers_books, con = engine)
publishers_books

,publisher,total_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


Больше всех выпустило книг издательство Penguin Books

In [12]:
#Определим автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
author_rating = '''SELECT
    authors.author,
    AVG(filtered.average_rating) AS avg_rating
    FROM
        (SELECT
            book_id AS book_id,
            COUNT(rating_id) AS rating_counts,
            AVG(rating) AS average_rating
        FROM
            ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) >= 50) 
        AS filtered
    INNER JOIN books ON books.book_id = filtered.book_id
    INNER JOIN authors ON authors.author_id = books.author_id
    GROUP BY authors.author
    ORDER BY avg_rating DESC
    LIMIT 10
'''
author_rating = pd.io.sql.read_sql(author_rating, con = engine)
author_rating

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


J.K. Rowling является автором с самым высоким рейтингом

In [13]:
#Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
mean_reviews = '''
        SELECT
        AVG(sub.count)
        FROM
        (SELECT COUNT(*)
            FROM reviews
            WHERE username IN
                (SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating) > 50)
            GROUP BY username)
            AS sub;
        '''

mean_reviews = pd.io.sql.read_sql(mean_reviews, con = engine)
mean_reviews

,avg
0,24.333333


В среднем пользователи оставляют 24 рецензии

## Шаг 3. Вывод по сделанной работе

- После 1 января 2000 года вышло 819 книг. 
- Издательство Penguin Books является лидером выпущенному количеству книг с более чем 50 страницами. Это неудивительно, так как оно является одним из самых популярных издательств на зарубежном книжном рынке.
- Джоан Роулинг является самым высокооценным автором из списка. 
- Пользователи, поставившие более 50 оценок, в среднем оставляют 24 рецензии. 